# Python Script to convert flight logs in .ulg format to .kml format for using them in GQIS

# **1. Connect to Google Drive to import files**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1.1 Import files

In [ ]:
################################IMPORTAR EL dataset entero a colab UNA VEZ TIENES LA ESTRUCTURA  ####################################################
####################################################################################


# Copiar el archivo data.yaml desde Drive a la ruta deseada en Colab
!cp -r "/content/drive/MyDrive/rinder_van_logs_1_2_6_25/Yoda_VM_logs_Day_2" "/content/rinder"
#"/content/dataset/lote_4"

# Verificar que se copió correctamente
!ls -l "/content/rinder"

total 1299788
-rw------- 1 root root 119579260 Jun  2 19:57 07_48_53.ulg
-rw------- 1 root root 132231113 Jun  2 19:57 08_44_19.ulg
-rw------- 1 root root 125581659 Jun  2 19:57 09_43_22.ulg
-rw------- 1 root root 114195985 Jun  2 19:57 10_44_08.ulg
-rw------- 1 root root 121857669 Jun  2 19:58 11_40_31.ulg
-rw------- 1 root root 143318709 Jun  2 19:58 12_36_29.ulg
-rw------- 1 root root 147294008 Jun  2 19:58 13_39_46.ulg
-rw------- 1 root root 146838158 Jun  2 19:58 14_44_59.ulg
-rw------- 1 root root 150440680 Jun  2 19:58 15_49_31.ulg
-rw------- 1 root root 129590103 Jun  2 19:58 16_57_07.ulg


# 1.2 Install necessary packages

In [ ]:
!pip install pyulog simplekml pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.3 MB/s eta 0:00:00
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65860 sha256=538d0f32ea0f67466d3ad8d75e34ba28b0bc800ee8efc46dfd99cba5f7671503
  Stored in directory: /root/.cache/pip/wheels/72/3e/80/c3e5c354c3cbe62d8c5e4fb63d9e7cdccc7f93399997ae465f
Successfully built simplekml


# **1.3. Transform .ulg to .kml**

Customizable parameters:

- Input directory for .ulg files

- Output directory for .kml files

- Output directory for .csv files

In [ ]:
import os
from pyulog import ULog, ulog2kml, ulog2csv
import pandas as pd
import simplekml
import shutil

# === CONFIGURA TUS RUTAS ===
entrada_dir = "/content/rinder"  # 📂 Carpeta con tus archivos .ulg (por ejemplo subida a Colab)
salida_kml_dir = "/content/rinder/kml"
salida_csv_dir = "/content/rinder"

os.makedirs(salida_kml_dir, exist_ok=True)
os.makedirs(salida_csv_dir, exist_ok=True)

# === PROCESAR TODOS LOS ARCHIVOS .ULG ===
for archivo in os.listdir(entrada_dir):
    if not archivo.lower().endswith(".ulg"):
        continue

    ruta_ulg = os.path.join(entrada_dir, archivo)
    nombre_base = os.path.splitext(archivo)[0]
    ruta_kml = os.path.join(salida_kml_dir, nombre_base + ".kml")

    print(f"\n📡 Procesando: {archivo}")

    # === Intento directo ===
    try:
        ulog2kml.convert_ulog2kml(ruta_ulg, ruta_kml)
        print(f"✅ KML generado directamente: {ruta_kml}")
        continue
    except Exception as e:
        print(f"⚠️ ulog2kml falló: {e}")

    # === Intento alternativo con CSV ===
    try:
        ulog = ULog(ruta_ulg)
        gps_msgs = [msg for msg in ulog.data_list if msg.name == "vehicle_gps_position"]
        if not gps_msgs or len(gps_msgs[0].data["lat"]) == 0:
            print(f"❌ Sin datos GPS en: {archivo}")
            continue

        subdir_csv = os.path.join(salida_csv_dir, nombre_base)
        os.makedirs(subdir_csv, exist_ok=True)

        ulog2csv.convert_ulog2csv(
            ruta_ulg,
            "vehicle_gps_position",  # ✅ usar string
            subdir_csv,
            ',',
            0,
            0
        )

        archivos_csv = os.listdir(subdir_csv)
        gps_file_list = [f for f in archivos_csv if "vehicle_gps_position" in f and f.endswith(".csv")]
        if not gps_file_list:
            print(f"❌ No se encontró archivo GPS en: {subdir_csv}")
            continue

        gps_csv_path = os.path.join(subdir_csv, gps_file_list[0])
        df = pd.read_csv(gps_csv_path)

        if len(df) == 0 or "lat" not in df.columns or "lon" not in df.columns:
            print(f"❌ CSV sin datos válidos: {gps_csv_path}")
            continue

        # ✅ Corregir lat/lon
        df["lat_deg"] = df["lat"] / 1e7
        df["lon_deg"] = df["lon"] / 1e7

        # ✅ Crear .kml desde lat/lon corregidos
        kml = simplekml.Kml()
        coords = list(zip(df["lon_deg"], df["lat_deg"]))
        kml.newlinestring(name=nombre_base, coords=coords)
        kml.save(ruta_kml)

        print(f"✅ KML generado desde CSV corregido: {ruta_kml}")

        # 🔁 Limpieza opcional
        # shutil.rmtree(subdir_csv, ignore_errors=True)

    except Exception as e:
        print(f"❌ Error en fallback con CSV: {e}")



📡 Procesando: 10_44_08.ulg
⚠️ ulog2kml falló: list index out of range
✅ KML generado desde CSV corregido: /content/rinder/kml/10_44_08.kml

📡 Procesando: 07_48_53.ulg
✅ KML generado directamente: /content/rinder/kml/07_48_53.kml

📡 Procesando: 15_49_31.ulg
✅ KML generado directamente: /content/rinder/kml/15_49_31.kml

📡 Procesando: 08_44_19.ulg
✅ KML generado directamente: /content/rinder/kml/08_44_19.kml

📡 Procesando: 12_36_29.ulg
✅ KML generado directamente: /content/rinder/kml/12_36_29.kml

📡 Procesando: 13_39_46.ulg
✅ KML generado directamente: /content/rinder/kml/13_39_46.kml

📡 Procesando: 14_44_59.ulg
✅ KML generado directamente: /content/rinder/kml/14_44_59.kml

📡 Procesando: 16_57_07.ulg
✅ KML generado directamente: /content/rinder/kml/16_57_07.kml

📡 Procesando: 09_43_22.ulg
✅ KML generado directamente: /content/rinder/kml/09_43_22.kml

📡 Procesando: 11_40_31.ulg
✅ KML generado directamente: /content/rinder/kml/11_40_31.kml


# 1.4 Save the results in Google Drive

In [ ]:
# 10. Guardar los resultados del experimento

# Define la carpeta de destino en Google Drive
dest_folder = "/content/drive/MyDrive/rinder_van_logs_1_2_6_25"
!mkdir -p "{dest_folder}"  # El comando mkdir -p crea esa carpeta (y cualquier subcarpeta necesaria) si no existe.

# Copia la carpeta de resultados (ajusta el nombre según el que se haya generado)
!cp -r /content/rinder/kml "{dest_folder}/"  # Copiar los resultados del modelo en drive

# Comprueba que se copiaron los archivos (opcional)
!ls "{dest_folder}"

2025-06-01  2025-06-1  kml  Yoda_VM_logs_Day_2
